In [134]:
import pandas as pd
import numpy as np

# Load the CSV file, replacing commas with dots and ensuring all data is treated as floats
prices = pd.read_csv('prices.csv', sep=';', dtype=str)

# Replace commas with dots and convert the entire DataFrame to floats
prices = prices.replace({',': '.'}, regex=True).astype(float)

# Split the CSV into left-hand side and right-hand side starting from the 'SPLIT' column
split_index = prices.columns.get_loc('SPLIT')

left_hand_side = prices.iloc[:, :split_index]  # All columns to the left of 'SPLIT'
right_hand_side = prices.iloc[:, split_index + 1:]  # All columns to the right of 'SPLIT'

# Rename the columns to reflect day -255 to day -1 on the left, and day +1, +2, ... on the right
left_columns = [f"day -{i+1}" for i in range(len(left_hand_side.columns))]
right_columns = [f"day +{i+1}" for i in range(len(right_hand_side.columns))]

# Apply new column names to the left-hand and right-hand sides
left_hand_side.columns = left_columns
right_hand_side.columns = right_columns

# Function to remove NaN, shift values toward day -255, and pad near the 'SPLIT'
def clean_and_shift_inward(row):
    non_nan_values = row.dropna().values  # Drop NaN values
    padding = [np.nan] * (len(row) - len(non_nan_values))  # Padding on the right side of valid values
    cleaned_row = np.concatenate([padding, non_nan_values])  # Shift non-NaN values inward
    return pd.Series(cleaned_row, index=row.index)

# Apply the cleaning function to each row of the left-hand side
left_hand_side_cleaned = left_hand_side.apply(clean_and_shift_inward, axis=1)

# Initialize a list to store merged results
merged_data = []

# Iterate over rows and concatenate cleaned left-hand side rows with right-hand side rows
for left_row, right_row in zip(left_hand_side_cleaned.iterrows(), right_hand_side.iterrows()):
    _, left_values = left_row
    _, right_values = right_row
    
    # Concatenate the left-hand side row with the corresponding right-hand side row
    merged_row = pd.concat([left_values, right_values])
    
    merged_data.append(merged_row)

# Convert the merged data back to a DataFrame
merged_df = pd.DataFrame(merged_data, columns=left_columns + right_columns)

# Add the 'SPLIT' column back to the merged DataFrame at the correct location
merged_df.insert(split_index, 'SPLIT', prices['SPLIT'])

# Print or inspect the merged DataFrame
print("Merged Data:")
print(merged_df)

# Save the merged data to a new CSV file with the original headers
merged_df.to_csv('merged_data.csv', sep=';', index=False)


Merged Data:
     day -1  day -2    day -3     day -4     day -5     day -6     day -7  \
0       NaN     NaN       NaN        NaN   0.000000 -41.843975 -23.170734   
1       NaN     NaN       NaN  -2.682555  -2.297092  -1.724142   2.073360   
2       NaN     NaN       NaN  -6.504065  -3.478259   0.900900  -4.464274   
3       NaN     NaN -3.738332   0.970876  -1.923072  -3.284311  -3.740489   
4       NaN     NaN -7.692301   0.000000 -16.666668   0.000000  20.000004   
..      ...     ...       ...        ...        ...        ...        ...   
443     NaN     NaN       NaN   1.515150   4.477608  -1.428568   1.449275   
444     NaN     NaN  0.000000   0.000000   0.000000   0.000000  42.857136   
445     NaN     NaN  0.000000   0.000000   0.000000   0.000000   0.000000   
446     NaN     NaN       NaN  -2.777785  -5.714279  -8.666664  -3.782350   
447     NaN     NaN       NaN -22.068960  -2.654862   2.272725   2.222216   

        day -8     day -9    day -10  ...   day +245  day +246

/var/folders/3k/zwv52hz93g96b2yjn3w9fywm0000gn/T/ipykernel_53745/965525380.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  prices = prices.replace({',': '.'}, regex=True).astype(float)
